In [2]:
import argparse
import base64
from datetime import datetime
import os
import shutil

import numpy as np
import socketio
import eventlet
import eventlet.wsgi
from PIL import Image
from flask import Flask
from io import BytesIO

from keras.models import load_model
import h5py
from keras import __version__ as keras_version

sio = socketio.Server()
app = Flask(__name__)
model = None
prev_image_array = None


class SimplePIController:
    def __init__(self, Kp, Ki):
        self.Kp = Kp
        self.Ki = Ki
        self.set_point = 0.
        self.error = 0.
        self.integral = 0.

    def set_desired(self, desired):
        self.set_point = desired

    def update(self, measurement):
        # proportional error
        self.error = self.set_point - measurement

        # integral error
        self.integral += self.error

        return self.Kp * self.error + self.Ki * self.integral


controller = SimplePIController(0.1, 0.002)
set_speed = 9
controller.set_desired(set_speed)


@sio.on('telemetry')
def telemetry(sid, data):
    if data:
        # The current steering angle of the car
        steering_angle = data["steering_angle"]
        # The current throttle of the car
        throttle = data["throttle"]
        # The current speed of the car
        speed = data["speed"]
        # The current image from the center camera of the car
        imgString = data["image"]
        image = Image.open(BytesIO(base64.b64decode(imgString)))
        image_array = np.asarray(image)
        steering_angle = float(model.predict(image_array[None, :, :, :], batch_size=1))

        throttle = controller.update(float(speed))

        print(steering_angle, throttle)
        send_control(steering_angle, throttle)

        # save frame
        if args.image_folder != '':
            timestamp = datetime.utcnow().strftime('%Y_%m_%d_%H_%M_%S_%f')[:-3]
            image_filename = os.path.join(args.image_folder, timestamp)
            image.save('{}.jpg'.format(image_filename))
    else:
        # NOTE: DON'T EDIT THIS.
        sio.emit('manual', data={}, skip_sid=True)


@sio.on('connect')
def connect(sid, environ):
    print("connect ", sid)
    send_control(0, 0)


def send_control(steering_angle, throttle):
    sio.emit(
        "steer",
        data={
            'steering_angle': steering_angle.__str__(),
            'throttle': throttle.__str__()
        },
        skip_sid=True)


if __name__ == '__main__':

    # check that model Keras version is same as local Keras version
    model_file = 'model.h5'
    # f = h5py.File(args.model, mode='r')
    f = h5py.File(model_file, mode='r')    
    model_version = f.attrs.get('keras_version')
    keras_version = str(keras_version).encode('utf8')

    if model_version != keras_version:
        print('You are using Keras version ', keras_version,
              ', but the model was built using ', model_version)

    model = load_model(model_file)
    recored_images_folder = 'recorded_images'
    if recored_images_folder != '':
        print("Creating image folder at {}".format(recored_images_folder))
        if not os.path.exists(recored_images_folder):
            os.makedirs(recored_images_folder)
        else:
            shutil.rmtree(recored_images_folder)
            os.makedirs(recored_images_folder)
        print("RECORDING THIS RUN ...")
    else:
        print("NOT RECORDING THIS RUN ...")

    # wrap Flask application with engineio's middleware
    app = socketio.Middleware(sio, app)

    # deploy as an eventlet WSGI server
    eventlet.wsgi.server(eventlet.listen(('', 4567)), app)


Creating image folder at recorded_images
RECORDING THIS RUN ...
connect  0079875296a0477f99a85b9cee0b4a53


(3722) wsgi starting up on http://0.0.0.0:4567
(3722) accepted ('127.0.0.1', 34387)


0.029730308800935745 0.918
connect  3d54a214b0a7414f913d7fce8d1517da
0.029631203040480614 0.936
connect  e3660612bc684263a48eeeae70e6351f


Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/eventlet/wsgi.py", line 507, in handle_one_response
    result = self.application(self.environ, start_response)
  File "/usr/local/lib/python3.5/dist-packages/engineio/middleware.py", line 47, in __call__
    return self.engineio_app.handle_request(environ, start_response)
  File "/usr/local/lib/python3.5/dist-packages/socketio/server.py", line 353, in handle_request
    return self.eio.handle_request(environ, start_response)
  File "/usr/local/lib/python3.5/dist-packages/engineio/server.py", line 252, in handle_request
    transport, b64)
  File "/usr/local/lib/python3.5/dist-packages/engineio/server.py", line 374, in _handle_connect
    ret = s.handle_get_request(environ, start_response)
  File "/usr/local/lib/python3.5/dist-packages/engineio/socket.py", line 89, in handle_get_request
    start_response)
  File "/usr/local/lib/python3.5/dist-packages/engineio/socket.py", line 136, in _upgrade_websocket


0.02964184246957302 0.9330696
connect  57fbc1f57c044b768d1955f0f98bb240
0.02686900459229946 0.8899895999999999
connect  937ae8d6526449fdb4651018e7728cc3


...

Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/eventlet/wsgi.py", line 507, in handle_one_response
    result = self.application(self.environ, start_response)
  File "/usr/local/lib/python3.5/dist-packages/engineio/middleware.py", line 47, in __call__
    return self.engineio_app.handle_request(environ, start_response)
  File "/usr/local/lib/python3.5/dist-packages/socketio/server.py", line 353, in handle_request
    return self.eio.handle_request(environ, start_response)
  File "/usr/local/lib/python3.5/dist-packages/engineio/server.py", line 252, in handle_request
    transport, b64)
  File "/usr/local/lib/python3.5/dist-packages/engineio/server.py", line 374, in _handle_connect
    ret = s.handle_get_request(environ, start_response)
  File "/usr/local/lib/python3.5/dist-packages/engineio/socket.py", line 89, in handle_get_request
    start_response)
  File "/usr/local/lib/python3.5/dist-packages/engineio/socket.py", line 136, in _upgrade_websocket


-0.017858337610960007 0.5654355999999994
connect  52e6792baf4d43c48acff2b2a6822c96
-0.009270588867366314 0.5650679999999995
connect  ab27b1cd77ff4620be95bba2d9bb9256


Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/eventlet/wsgi.py", line 507, in handle_one_response
    result = self.application(self.environ, start_response)
  File "/usr/local/lib/python3.5/dist-packages/engineio/middleware.py", line 47, in __call__
    return self.engineio_app.handle_request(environ, start_response)
  File "/usr/local/lib/python3.5/dist-packages/socketio/server.py", line 353, in handle_request
    return self.eio.handle_request(environ, start_response)
  File "/usr/local/lib/python3.5/dist-packages/engineio/server.py", line 252, in handle_request
    transport, b64)
  File "/usr/local/lib/python3.5/dist-packages/engineio/server.py", line 374, in _handle_connect
    ret = s.handle_get_request(environ, start_response)
  File "/usr/local/lib/python3.5/dist-packages/engineio/socket.py", line 89, in handle_get_request
    start_response)
  File "/usr/local/lib/python3.5/dist-packages/engineio/socket.py", line 136, in _upgrade_websocket


-0.029118819162249565 0.5629705999999997


Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/eventlet/wsgi.py", line 507, in handle_one_response
    result = self.application(self.environ, start_response)
  File "/usr/local/lib/python3.5/dist-packages/engineio/middleware.py", line 47, in __call__
    return self.engineio_app.handle_request(environ, start_response)
  File "/usr/local/lib/python3.5/dist-packages/socketio/server.py", line 353, in handle_request
    return self.eio.handle_request(environ, start_response)
  File "/usr/local/lib/python3.5/dist-packages/engineio/server.py", line 252, in handle_request
    transport, b64)
  File "/usr/local/lib/python3.5/dist-packages/engineio/server.py", line 374, in _handle_connect
    ret = s.handle_get_request(environ, start_response)
  File "/usr/local/lib/python3.5/dist-packages/engineio/socket.py", line 89, in handle_get_request
    start_response)
  File "/usr/local/lib/python3.5/dist-packages/engineio/socket.py", line 136, in _upgrade_websocket
